In [2]:
class MyFunctor f where
  fmap' :: (a -> b) -> f a -> f b

instance MyFunctor ((->) a) where
  -- fmap' f g = \x -> f (g x)
  -- fmap' f g x = f (g x)
  -- fmap' f g = f . g
  fmap' = (.)

Die Funktion `(fmap . fmap)` ist eine elegante Möglichkeit, eine Funktion auf den innersten Wert in einem zweifach faltbaren Kontext anzuwenden. Sie erlaubt uns, den inneren Listenkontext zu ignorieren und sich nur auf den Wert in den innersten Listen zu konzentrieren.

## Reine Tyen verschiedener Funktoren und Funktionen
```haskell
fmap' test (+1)         :: Int -> Int
fmap'                   :: MyFunctor f              => (a -> b) -> f a -> f b
fmap' test              :: MyFunctor f              => f String -> f String
fmap' id                :: MyFunctor f              => f b -> f b
fmap' (test . test2)    :: MyFunctor f              => f String -> f String
fmap' test2 . fmap test :: (MyFunctor f, Functor f) => f String -> f String
fmap . fmap             :: (Functor f1, Functor f2) => (a -> b) -> f1 (f2 a) -> f1 (f2 b)
fmap fmap fmap          :: (Functor f1, Functor f2) => (a -> b) -> f1 (f2 a) -> f1 (f2 b)
```

## Gleichheit `fmap' id == id`
```haskell
    fmap' id == id              (1)
=>  fmap (\x -> x) == (\x -> x) (2)
=>  fmap' id f x === id (f x)   (3)


    fmap id f                   (4) 
=>  \x -> id (f x)              (5)
=>  \x -> f x                   (6)
=>  f                           (7)
=== \x -> id x                  (8)
=>  id                          (9)
```

Da id nur den Wert des eingegeben Paramters zurückgibt, kann diese in Gleichung (5) eliminiert werden. Durch Komposition ergibt sich dann als Ergebnis nur die Funktion, welche als 2. parameter in fmap eingegeben wird. Dies ist gleichzusetzen mit `(id)`.


```haskell
fmap' (f . g) == fmap' f . fmap' g
```

## Gleichheit `fmap fmap fmap == fmap . fmap`

```haskell 
    fmap1 fmap2 fmap3
    fmap3 :: (a -> b) -> (f a -> f b)
=>  fmap3 :: (p       -> q)
=>  fmap3 :: (->) p q mit p == (a->b) und q == (f a->f b)
```

da `fmap (->) p` dasselbe ist wie `(.)` (siehe konstruktor oben), kann der Ausdruck `fmap fmap fmap` reduziert werden auf `(.) fmap fmap`

---

## Konstruktion des Typs `fmap . fmap`
```haskell
(.)  ::                 (   a       ->      b) -> (r -> a) -> (r -> b)                        (0)
fmap :: Functor f =>    (c -> d)    -> (f c -> f d)
fmap :: Functor g =>    (x -> y)    -> (g x -> g y)
```
```haskell
    fmap :: Functor f =>    (c -> d)    -> (f c -> f d)                         (0.1)
==                              a       ->      b                               (0.2)
```
---
```haskell
    fmap :: Functor g =>    (x -> y)    -> (g x -> g y)                         (1.1)
=>                              r       -> (c   -> d)                           (1.2)
ersetzen von a und b, kombinieren von (0) und (0.1)
ZWISCHENERGEBNIS
===> ((c -> d) -> (f c -> f d)) -> (r -> (c -> d)) -> (r -> (f c -> f d)) <===  (2.1)
          a    ->       b       -> (r ->    a    ) -> (r ->      b)             (2.2)
WEITERRECHNEN
da (r -> (c -> d)) !== (x -> y)    -> (g x -> g y) sein muss, wird 
=> r == (x-> y), c == (g x), d == (g y)                                         (3.1)

ersetzen von c und d, kombinieren von (2.1) und (3.1)
    (( c  ->  d ) -> (f   c   -> f   d  )) -> | ((   r    ->   c   -> d )) -> |    r     -> f  c    -> f   d
==  ((g x -> g y) -> (f (g x) -> f (g y))) -> | ((x -> y) -> (g x -> g y)) -> | (x -> y) -> f (g x) -> f (g y)
==                  fmap                      |             fmap               |            ERGEBNIS
damit ist das ergebnis von fmap . fmap
(x -> y) -> f (g x) -> f (g y)

```

## Verwendung von `fmap . fmap`

In [1]:
test :: String -> String
test a = a ++ " from a"

testList :: [String]
testList = ["Hello", "World", "Its", "me"]

test2 :: Char -> Char
test2 a = 'x'


result :: [String]
result = fmap fmap fmap test2 testList

result

["xxxxx","xxxxx","xxx","xx"]

**Test fmap, funktioniert leider nicht**
```haskell
testList3 :: [[[[String]]]]
testList3 = [[[["Hello", "World"], ["Open", "AI"]], [["Test"], ["123"]]], [[["Hello", "World"], ["Open", "AI"]], [["Test"], ["123"]]]]

test4 :: Char -> Char
test4 x = 'x'


result :: [[[[String]]]]
result = fmap fmap fmap fmap fmap fmap test4 testList3

```

In [5]:
maybeTest :: [Maybe Int]
maybeTest = [Just 1, Just 2, Just 3]

(fmap . fmap) (+1) maybeTest


[Just 2,Just 3,Just 4]

In [6]:
data Zero a
data One a = One
data Identity a = Identity a
data Const a b = Const a
data Square a = Square a a
data Product a b = Product a b

Line 3: Use newtype instead of data
Found:
data Identity a = Identity a
Why not:
newtype Identity a = Identity aLine 4: Use newtype instead of data
Found:
data Const a b = Const a
Why not:
newtype Const a b = Const a

In [8]:
instance MyFunctor Zero where
    fmap' _ _ = undefined

instance MyFunctor One where
    fmap' f One = One

instance MyFunctor Identity where
    fmap' f (Identity a) = Identity (f a)

instance MyFunctor (Const a) where
    fmap' f (Const a) = Const a

instance MyFunctor Square where
    fmap' f (Square a b) = Square (f a) (f b)

instance MyFunctor (Product a) where
    fmap' f (Product a b) = Product a (f b)